In this notebook, we show how we can train a model with PyTorch and save it as a TileDB array on TileDB-Cloud.
Firstly, let's import what we need and define some variables needed for training a model.

In [1]:
import os

import tiledb.cloud
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from tiledb.ml.models.pytorch import PyTorchTileDBModel

epochs = 1
batch_size_train = 128
learning_rate = 0.01
momentum = 0.5
log_interval = 10

# Set random seeds for anything using random number generation
torch.manual_seed(seed=1)

# Disable nondeterministic algorithms
torch.backends.cudnn.enabled = False

We then create a TileDB-Cloud context, log in to TileDB via our API token (or username/password) and get our username.

In [2]:
ctx = tiledb.cloud.Ctx()
tiledb.cloud.login(token=os.getenv('TILEDB_API_TOKEN'))
namespace = tiledb.cloud.client.default_user().username

We  will also need the DataLoaders API for the dataset. We will also employ TorchVision which let's as load the MNIST
dataset in a handy way. We'll use a batch_size of 64 for training while the values 0.1307 and 0.3081 used for
the Normalize() transformation below are the global mean and standard deviation of the MNIST dataset,
we'll take them as a given here.

In [3]:
data_home = os.path.join(os.path.pardir, os.path.pardir, 'data')
dataset = torchvision.datasets.MNIST(
    root=data_home, 
    train=True, 
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size_train, shuffle=True)

Moving on, we build our network. We'll use two 2-D convolutional layers followed by two fully-connected
layers. As activation function we'll choose ReLUs and as a means of regularization we'll use two dropout layers.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)


We will now initialise our Neural Network and optimizer.

In [5]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

We continue with the training loop and we iterate over all training data once per epoch. Loading the individual batches
is handled by the DataLoader. We need to set the gradients to zero using optimizer.zero_grad() since PyTorch by default
accumulates gradients. We then produce the output of the network (forward pass) and compute a negative log-likelihodd
loss between the output and the ground truth label. The backward() call we now collect a new set of gradients which we
propagate back into each of the network's parameters using optimizer.step().

In [6]:
train_losses = []
train_counter = []

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

for epoch in range(1, epochs + 1):
  train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.358812
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285138
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.306635
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.270880
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.236739
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.243352
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.183357
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.148380
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.104831
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.004480
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.861931
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.884068
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.797761
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.687489
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.509098
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.766841
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.469026
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.349417
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.291566
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.022800
Train Epoch: 1 [

We can move on by defining a TileDBPyTorch model and use model save functionality in order to save it directly to
our bucket on S3 (defined with AWS credentials in your TileDB-Cloud account) and register it on TileDB-Cloud.

In [7]:
print('Defining PyTorchTileDBModel model...')
# In order to save our model on S3 and register it on TileDB-Cloud we have to pass our Namespace and TileDB Context.
tiledb_model = PyTorchTileDBModel(uri='tiledb-pytorch-model', namespace=namespace, ctx=ctx, model=model, optimizer=optimizer)

# We will need the uri that was created from our model class
# (and follows pattern tiledb://my_username/s3://my_bucket/my_array),
# in order to interact with our model on TileDB-Cloud.
tiledb_cloud_model_uri = tiledb_model.uri

print('Saving model on S3 and registering on TileDB-Cloud...')
tiledb_model.save(meta={'epochs': epochs,
                        'train_loss': train_losses})


Defining PyTorchTileDBModel model...
Saving model on S3 and registering on TileDB-Cloud...


Finally, we can use TileDB-Cloud API as described in our [cloud documentation](https://docs.tiledb.com/cloud/), in order
to list our models, get information and deregister them.

In [8]:
# List all our models. Here, we filter with file_type = 'ml_model'. All machine learning model TileDB arrays are of type
# 'ml_model'
print(tiledb.cloud.client.list_arrays(file_type=['ml_model'], namespace=namespace))

# Get model's info
print(tiledb.cloud.array.info(tiledb_cloud_model_uri))

# Load our model for inference
# Place holder for the loaded model
loaded_model = Net()
loaded_optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

tiledb_model = PyTorchTileDBModel(uri=os.path.basename(tiledb_cloud_model_uri), namespace=namespace, ctx=ctx)
tiledb_model.load(model=loaded_model, optimizer=loaded_optimizer)

# Check model parameters
assert str(model.state_dict()) == str(loaded_model.state_dict())

# Check optimizer parameters
assert str(optimizer.state_dict()) == str(loaded_optimizer.state_dict())

{'arrays': [{'access_credentials_name': 'gsk',
             'allowed_actions': ['read',
                                 'read_array_logs',
                                 'read_array_info',
                                 'read_array_schema',
                                 'write',
                                 'edit'],
             'description': None,
             'file_properties': None,
             'file_type': 'ml_model',
             'id': '686687e4-f21f-42b6-90fc-464acf221a06',
             'is_favorite': None,
             'last_accessed': datetime.datetime(2022, 8, 10, 18, 9, 57, tzinfo=tzutc()),
             'license_id': None,
             'license_text': None,
             'logo': None,
             'name': 'tiledb-pytorch-model',
             'namespace': 'george.sakkis',
             'namespace_subscribed': None,
             'pricing': None,
             'public_share': False,
             'read_only': False,
             'share_count': None,
             'size'

In [9]:
# Deregister model and physically delete the array from the S3 bucket
tiledb.cloud.array.delete_array(tiledb_cloud_model_uri)